In [1]:
pip install refl1d


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install ipympl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
import os
import math
import json
import numpy as np

import refl1d
from refl1d.names import *

#import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline
import seaborn as sea

In [6]:

from scipy.interpolate import CubicSpline
import random
def calculate_reflectivity(q, model_description, q_resolution=0.0294855):
    """
        Reflectivity calculation using refl1d
    """
    zeros = np.zeros(len(q))
    dq = q_resolution * q / 2.355

    # The QProbe object represents the beam
    probe = QProbe(q, dq, data=(zeros, zeros))

    layers = model_description['layers']
    sample = Slab(material=SLD(name=layers[0]['name'],
                               rho=layers[0]['sld']), interface=layers[0]['roughness'])
    # Add each layer
    for l in layers[1:]:
        sample = sample | Slab(material=SLD(name=l['name'],
                               rho=l['sld'], irho=l['isld']),
                               thickness=l['thickness'], interface=l['roughness'])

    probe.background = Parameter(value=model_description['background'], name='background')
    expt = Experiment(probe=probe, sample=sample)

    q, r = expt.reflectivity()
    z, sld, _ = expt.smooth_profile()
    # print(z[0], z[-1])
    # z = np.insert(z, 0, -271)
    # sld = np.insert(sld, 0, 2.7)
    # z = np.append(z,z[-1] + 0.00000001)
    # sld = np.append(sld, 0)
    # z = np.append(z, 1750+0.00001)
    # sld = np.append(sld, 0)
    zNew = np.linspace(z[0], z[-1], num=900)
    # print(z[0], z[-1])
    newCurve = CubicSpline(z, sld)
    sldNew = []
    for i in range(zNew.shape[0]):
      sldNew.append(newCurve(zNew[i]))
    return model_description['scale'] * r, zNew, sldNew
class ReflectivityModels(object):
        # Neutrons come in from the last item in the list

    def __init__(self,q=None, name='polymer'):

        self.parameters = [
                  dict(i=0, par='roughness', bounds=[6, 9]),
                  # The following is the Si oxide layer
                  dict(i=1, par='sld', bounds=[3.47, 3.47]),
                  dict(i=1, par='thickness', bounds=[6, 10]),
                  dict(i=1, par='roughness', bounds=[3, 4.1]),

                  # The next three layers are the polymer
                  dict(i=2, par='sld', bounds=[1, 2]),
                  dict(i=2, par='thickness', bounds=[90, 180]),
                  dict(i=2, par='roughness', bounds=[0, 10]),
                  dict(i=3, par='sld', bounds=[1,2]),
                  dict(i=3, par='thickness', bounds=[40, 125]),
                  dict(i=3, par='roughness', bounds=[0, 1]),
                  # dict(i=4, par='sld', bounds=[0, 6.4]),
                  #dict(i=4, par='sld', bounds=[0, 2]),
                  #dict(i=4, par='thickness', bounds=[10, 300]),
                  #dict(i=4, par='roughness', bounds=[0, 10]),
                  # The next layer is the air
                  #dict(i=5, par='sld', bounds=[0, 0]),
                  #dict(i=5, par='thickness', bounds=[1, 50]),
                  #dict(i=5, par='roughness', bounds=[0, 10]),
                 ]
        self.model_description = dict(layers=[
                                dict(sld=2.07, isld=0, thickness=0, roughness=3, name='substrate'),
                                dict(sld=3.3, isld=0, thickness=16.2139, roughness=7, name='siox'),
                                dict(sld=1.65505, isld=0, thickness=144.106, roughness=9.71491, name='bottom'),
                                dict(sld=1.65505, isld=0, thickness=95.2721, roughness=1, name='bulk'),
                                # dict(sld=0.0, isld=0, thickness=1000, roughness=0, name='top_layer'),
                                dict(sld=0.0, isld=0, thickness=0, roughness=0, name='air')
                         ],
                         scale=1,
                         background=0,
                        )
        # The following are unmodified physical parameters and corresponding reflectivity data
        self._pars_array = []
        self._refl_array = []
        self._smooth_array = []
        # The following are the parameters, mapped between -1 and 1.
        self._train_pars = []
        self._train_data = None
        self._config_name = name

        if q is None:
            self.q = np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308)
        else:
            self.q = q

    @classmethod
    def from_dict(cls, pars, q_array=None):
        """
            Create ReflectivityModels object from a dict that
            defines the reflectivity model parameters and how
            the training set should be generated.
        """
        m = cls(q_array, name=pars['name'])
        m.model_description =  pars['model']
        m.parameters = pars['parameters']
        return m

    def generate(self, n=100):
        """
            Generate a random sample of models
        """
        npars = len(self.parameters)
        self._train_pars = np.random.uniform(low=-1, high=1, size=[n, npars])
        # print(self.parameters, len(self.parameters))
        # Compute model parameters and reflectivity using these values
        self.compute_reflectivity()

    def to_model_parameters(self, pars):
        """
            Transform an array of parameters to a list of calculable models
        """
        pars_array = np.zeros(pars.shape)

        for i, par in enumerate(self.parameters):
            a = (par['bounds'][1]-par['bounds'][0])/2.
            b = (par['bounds'][1]+par['bounds'][0])/2.
            pars_array.T[i] = pars.T[i] * a + b

        return pars_array

    def compute_reflectivity(self):
        """
            Transform an array of parameters to a list of calculable models
            and compute reflectivity
        """
        print("Computing reflectivity")
        self._pars_array = self.to_model_parameters(self._train_pars)
        # print(len(self._train_pars), len(self._pars_array))
        # Compute reflectivity
        # count = 0
        # print(self._pars_array, "hi", self._pars_array[1][3])
        # self.tempPars = np.copy(self._pars_array)
        # self.tempTrain = np.copy(self._train_pars)
        # print(self.tempPars, "tempPars")
        # deleted = 0
        # for j in range(len(self._pars_array)):
        #   # print(j, self._pars_array[1][3])
        #   # print(self._pars_array[j][3], j, "hello?")
        #   if self._pars_array[j][3] > self._pars_array[j][2]*0.3 or self._pars_array[j][6] > self._pars_array[j][5]*0.3 or self._pars_array[j][9] > self._pars_array[j][8]*0.3 or self._pars_array[j][12] > self._pars_array[j][11]*0.3:
        #     self.tempPars = np.delete(self.tempPars, j-deleted, axis=0)
        #     self.tempTrain = np.delete(self.tempTrain, j-deleted, axis=0)
        #     deleted += 1
        # self._pars_array = self.tempPars
        # print(deleted)
        # self._train_pars = self.tempTrain
        # print(len(self.tempPars), "second temppars")
        # print(self._pars_array, "pars array")
        for p in self._pars_array:
            # print(p)
            _desc = self.get_model_description(p)
            r, z, sld = calculate_reflectivity(self.q, _desc)
            # # print(_desc)
            # # print(_desc)
            # tooBig = False
            # # print(_desc)
            # # for j in range(len(_desc['layers'])):
            #   # print(_desc)
            # if _desc['layers'][2]["roughness"] >= _desc['layers'][3]["thickness"] or _desc['layers'][3]["roughness"] >= _desc['layers'][2]["thickness"] or _desc['layers'][4]["roughness"] >= _desc['layers'][3]["thickness"] or _desc['layers'][3]["roughness"] >= _desc['layers'][3]["thickness"] or _desc['layers'][2]["roughness"] >= _desc['layers'][2]["thickness"]:
            #   continue
            # else:
                # print('HI')
            self._refl_array.append(r)
            self._smooth_array.append([z,sld])
            # isNegative = False
            # for j in range(len(sld)):
            #   if sld[j] < 0:
            #     isNegative = True
            #     break
            # if not isNegative:
            #   self._refl_array.append(r)
            #   self._smooth_array.append([z,sld])
            # if isNegative:
            #   self._pars_array = np.delete(self._pars_array, count)
            #   self._train_pars = np.delete(self._train_pars,count)
            # count += 1

    def get_model_description(self, pars):
        """
            Convert the parameter list to a model description that we can use
            to compute R(q).
        """
        for i, par in enumerate(self.parameters):
            self.model_description['layers'][par['i']][par['par']] = pars[i]
        return self.model_description

    def get_preprocessed_data(self, errors=None):
        """
            Pre-process data
            If errors is provided, a random error will be added, taking the errors array
            as a relative uncertainty.
        """
        if errors is None:

            self._train_data = np.log10(self._refl_array*self.q**2/self.q[0]**2)
        else:
            _data = self._refl_array * (1.0 + np.random.normal(size=len(errors)) * errors)
            # Catch the few cases where we generated a negative intensity and take
            # the absolute value
            _data[_data<0] = np.fabs(_data[_data<0])
            self._train_data = np.log10(_data*self.q**2/self.q[0]**2)

        return self._train_pars, self._train_data

    def save(self, output_dir=''):
        """
            Save all data relevant to a training set
            @param output_dir: directory used to store training sets
        """
        # Save q values
        np.save(os.path.join(output_dir, "%s_q_values" % self._config_name), self.q)

        # Save training set
        if self._train_data is not None:
            np.save(os.path.join(output_dir, "%s_data" % self._config_name), self._train_data)
            np.save(os.path.join(output_dir, "%s_pars" % self._config_name), self._train_pars)

    def load(self, data_dir=''):
        self.q = np.load(os.path.join(data_dir, "%s_q_values.npy" % self._config_name))
        self._train_data = np.load(os.path.join(data_dir, "%s_data.npy" % self._config_name))
        self._train_pars = np.load(os.path.join(data_dir, "%s_pars.npy" % self._config_name))
        return self.q, self._train_data, self._train_pars


In [7]:
m = ReflectivityModels()
m.generate(30000)
pars, train_data = m.get_preprocessed_data()
fig = plt.figure(dpi=100, figsize=[8,8])
# Let's look at a few curves
min_x = float('inf')
max_x = -float('inf')
for index in range(len(m._smooth_array)):
  min_x = min(m._smooth_array[index][0])
  for i in range(len(m._smooth_array[index][0])):
    m._smooth_array[index][0][i] -= min_x
# for index in range(len(m._smooth_array)):
#     plt.plot(m._smooth_array[index][0], m._smooth_array[index][1])
# plt.show()
settingUp = []
SLDSet = []
for i in range(len(m._smooth_array)):
  settingUp.append(np.array([m.q,m._refl_array[i]]))
  SLDSet.append(np.array(m._smooth_array[i]))
  # print(np.array(m._smooth_array[i]).shape)

totalStack = np.stack(settingUp)
totalParams = np.stack(SLDSet)
print(totalStack.shape, totalParams.shape)
np.save("SLD_CurvesPoly11.npy", totalParams)
np.save("NR-SLD_CurvesPoly11.npy", totalStack)

Computing reflectivity
(10000, 2, 308) (10000, 2, 900)


<Figure size 800x800 with 0 Axes>

In [ ]:
# NOTE: Here I would pass the q-array of the time-resolved data when instantiating this object.

count = 0
# Generate 100 curves
for first in range(4,5):
    for second in range(1,6):
        # count += 1
        # if count <= 2:
        #     continue
        print(first, second)
        m = ReflectivityModels(first, second)
        m.generate(30000)
        pars, train_data = m.get_preprocessed_data()
        fig = plt.figure(dpi=100, figsize=[8,8])
        # Let's look at a few curves
        min_x = float('inf')
        max_x = -float('inf')
        for index in range(len(m._smooth_array)):
          min_x = min(m._smooth_array[index][0])
          for i in range(len(m._smooth_array[index][0])):
            m._smooth_array[index][0][i] -= min_x
        # for index in range(len(m._smooth_array)):
        #     plt.plot(m._smooth_array[index][0], m._smooth_array[index][1])
        # plt.show()
        settingUp = []
        SLDSet = []
        for i in range(len(m._smooth_array)):
          settingUp.append(np.array([m.q,m._refl_array[i]]))
          SLDSet.append(np.array(m._smooth_array[i]))
          # print(np.array(m._smooth_array[i]).shape)

        totalStack = np.stack(settingUp)
        totalParams = np.stack(SLDSet)
        print(totalStack.shape, totalParams.shape)
        np.save("SLD_CurvesPoly" + str(first) + str(second) + ".npy", totalParams)
        np.save("NR-SLD_CurvesPoly" + str(first) + str(second) + ".npy", totalStack)



In [ ]:
pars, train_data = m.get_preprocessed_data()


fig = plt.figure(dpi=100, figsize=[8,8])
# Let's look at a few curves
min_x = float('inf')
max_x = -float('inf')
# print(m._refl_array)
# for index in range(6000):
#     # You can access the true physical parameters by transforming the pre-processed parameters
#     # which are between -1 and 1.
#     model_parameters = m.to_model_parameters(m._train_pars[index])

#     # print("Parameters: %s" % model_parameters)
#     # model_parameters = m.to_model_parameters(m._train_pars[index])

#     # print("Parameters: %s" % model_parameters)

#     # You can access the unmodified reflectivity curve by using m._refl_array directly.
#     min_x = min(min_x, min(m._smooth_array[index][0]))
#     max_x = max(max_x, max(m._smooth_array[index][0]))

# for index in range(18000):
#   for j in range(len(m._smooth_array[index][0])):
#     if m._smooth_array[index][1][j] == 0:
#       max_x = max(max_x, m._smooth_array[index][0][j])
#       break

# print(max_x)
for index in range(len(m._smooth_array)):
  min_x = min(m._smooth_array[index][0])
  for i in range(len(m._smooth_array[index][0])):
    m._smooth_array[index][0][i] -= min_x
  # zNew = np.linspace(m._smooth_array[index][0][0], 800, num=308)
  # newCurve = CubicSpline(m._smooth_array[index][0], m._smooth_array[index][1])
  # sldNew = []
  # for i in range(zNew.shape[0]):
  #   sldNew.append(newCurve(zNew[i]))
  # m._smooth_array[index][0] = zNew
  # m._smooth_array[index][1] = sldNew
# for index in range(80000):
#   print(m._smooth_array[index][0][0], m._smooth_array[index][1][0])
for index in range(len(m._smooth_array)):
    plt.plot(m._smooth_array[index][0], m._smooth_array[index][1])
plt.show()

In [ ]:
settingUp = []
SLDSet = []
for i in range(len(m._smooth_array)):
  settingUp.append(np.array([m.q,m._refl_array[i]]))
  SLDSet.append(np.array(m._smooth_array[i]))
  # print(np.array(m._smooth_array[i]).shape)

totalStack = np.stack(settingUp)
totalParams = np.stack(SLDSet)
print(totalStack.shape, totalParams.shape)
np.save("SLD_CurvesPoly" + str(first) + str(second) + ".npy", totalParams)
np.save("NR-SLD_CurvesPoly" + str(first) + str(second) + ".npy", totalStack)

In [ ]:
def calculate_reflectivity_from_profile(q, z_step, sld, q_resolution=0.0294855):

    """

        Reflectivity calculation using refl1d from an array of microslabs

    """

    sld = np.flip(sld)
    zeros = np.zeros(len(q))
    dq = q_resolution * q / 2.355
    # The QProbe object represents the beam

    probe = QProbe(q, dq, data=(zeros, zeros))
    sample = Slab(material=SLD(name='back', rho=sld[0], irho=0), interface=0)
    # Add each layer
    _prev_z = z_step[0]
    for i, _sld in enumerate(sld):
        if i>0:
            thickness = z_step[i] - _prev_z
            sample = sample | Slab(material=SLD(name='l_%d' % i, rho=_sld, irho=0),
                                                thickness=thickness,
                                                interface=0)
        _prev_z = z_step[i]
    probe.background = Parameter(value=0, name='background')
    expt = Experiment(probe=probe, sample=sample)
    a, r = expt.reflectivity()
    return a, r
z, sld = m._smooth_array[0][0], m._smooth_array[0][1]
plt.plot(z, sld)
plt.show()

a, r = calculate_reflectivity_from_profile(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308), z, sld)
# r = np.log10(r)
# np.save("File 202356_NR.npy", [a,r])

# for i in range(len(r)):
#     # x_pointsPoly[i][j] -= min_valXNR
#     r[i] -= min_valYNR
#     # x_pointsPoly[i][j] /= (max_valXNR - min_valXNR)
#     r[i] /= (max_valYNR - min_valYNR)
# for i in range(len(train_datasetPoly)):
#     data, label = train_datasetPoly[i]
#     # print(label[0])
#     if label[0] == 202356:
#         # print("hi")
#         # print(data[0])
#         # for j in range(len(r)):
#         #     r -= min_valYNR
#         #     r /= (max_valYNR - min_valYNR)
#         # print("hi")
#         # print(data[0])
# print(m._refl_array[0])
plt.scatter(m.q,m._refl_array[0], label = "Original NR curve")
plt.plot(m.q,r, label = "Reproduced NR curve from SLD profile", c='r')
plt.legend()
# plt.xscale("log")
plt.xlabel("q [1/Å]")
plt.ylabel("r [Å]")
plt.title("Neutron Reflectivity")

plt.yscale("log")
plt.savefig('Example SLD to NR')
# print(data)
plt.show()
# print("h9i")

In [ ]:
z, sld = np.load("File 202413_2d (3).npy")
plt.style.use("seaborn-darkgrid")
plt.plot(z, sld, linewidth=4, color="g")
plt.xlabel("z [Å]")
plt.ylabel("SLD [$10^{-6}Å^{-2}$]")
plt.title("Scattering Length Density Profile")
plt.savefig('202413 SLD Graph.jpeg')

plt.show()
a, r = calculate_reflectivity_from_profile(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308), z, sld)
other = np.load("./Neutron_Reflect/npoutputsPoly/np_out_REFL_202413_combined_data_auto.npy")
# plt.plot(m.q,)
yPoints = []
for i in range(len(other)):
    yPoints.append(other[i][1])
plt.plot(m.q,r, linewidth=2, color='r', label="Reproduced NR curve from SLD profile")
plt.scatter(m.q, yPoints, linewidth=0.01, label="Original Experimental NR curve")
# plt.xscale("log")
plt.xlabel("q [1/Å]")
plt.ylabel("r [Å]")
plt.title("Neutron Reflectivity")
plt.yscale("log")
plt.legend()
# print(data)
plt.savefig('202413 Neutron Reflectivity Graph.jpeg')
plt.show()

